# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

  Using cached geoviews-1.12.0-py3-none-any.whl (557 kB)
  Using cached bokeh-3.4.1-py3-none-any.whl (7.0 MB)
  Using cached Cartopy-0.23.0-cp310-cp310-win_amd64.whl (10.9 MB)
  Using cached shapely-2.0.4-cp310-cp310-win_amd64.whl (1.4 MB)
  Using cached holoviews-1.18.3-py2.py3-none-any.whl (4.3 MB)
  Using cached pyproj-3.6.1-cp310-cp310-win_amd64.whl (6.1 MB)
  Using cached panel-1.4.1-py3-none-any.whl (24.7 MB)
  Using cached contourpy-1.2.1-cp310-cp310-win_amd64.whl (187 kB)
  Using cached pyshp-2.3.1-py2.py3-none-any.whl (46 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached linkify_it_py-2.0.3-py3-none-any.whl (19 kB)
  Using cached param-2.1.0-py3-none-any.whl (116 kB)
  Using cached mdit_py_plugins-0.4.0-py3-none-any.whl (54 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Attempting uninstall: param
    Found existing installation: param 1.12.3
    Uninstalling param-1.12.3:
      Successfully uninstalled param-1.12.3
  Attempting unins

ImportError: cannot import name 'PandasAPI' from 'holoviews.core.data' (C:\Users\jack\anaconda3\lib\site-packages\holoviews\core\data\__init__.py)

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display
#could not make the map show

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
temp = city_data_df.loc[(city_data_df["Max Temp"]> 15) & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] <20) ,:]

# Drop any rows with null values
vacation_df = temp.dropna(how="any")

# Display sample data
vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,saint-louis,16.3333,-15.0000,38.80,24,4,1.78,SN,1666108247
42,42,margate,51.3813,1.3862,16.84,76,3,1.34,GB,1666108256
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
53,53,lasa,46.6166,10.7002,23.49,68,11,2.01,IT,1666108019


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame().assign(City=vacation_df["City"], Country= vacation_df["Country"], Lat=vacation_df["Lat"],Lng=vacation_df["Lng"],Humidity=vacation_df["Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.sample(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
276,tura,IN,25.5198,90.2201,90,
375,port hedland,AU,-20.3167,118.5667,73,
516,guerrero negro,MX,27.9769,-114.0611,68,
391,saint-francois,FR,46.4154,3.9054,57,
128,half moon bay,US,37.4636,-122.4286,86,
481,abu zabad,SD,12.3500,29.2500,29,
451,san joaquin,US,37.9333,-121.3011,89,
340,santa rosalia,MX,27.3167,-112.2833,56,
125,jashpurnagar,IN,22.9000,84.1500,87,
317,salalah,OM,17.0151,54.0924,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
import json
# Set parameters to search for a hotel
radius = 10000
api = "2c9f4d3ffaf847218390086057516cd3"
category = "accommodation.hotel"
params = {"categories": category,
   "apiKey":api
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params).json() #chatGpt give me the direct conversion with .json()
    
    # Convert the API response to JSON format
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-louis - nearest hotel: No hotel found
margate - nearest hotel: The Reading Rooms
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
lasa - nearest hotel: Hotel "Drei Kreuz"
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
jashpurnagar - nearest hotel: No hotel found
half moon bay - nearest hotel: Half Moon Bay Inn
puerto del rosario - nearest hotel: Hotel Tamasite
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
buala - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
tura - nearest hotel: Hotel Polo Orchid
alice springs - nearest hotel: Aurora Alice Springs
kingman - nearest hotel: Home2 Suites by Hilton Kingman
horsham - nearest hotel: Premier Inn
salalah - nearest hotel: Muscat International Hotel
harwich - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
29,saint-louis,SN,16.3333,-15.0000,24,No hotel found
42,margate,GB,51.3813,1.3862,76,The Reading Rooms
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
53,lasa,IT,46.6166,10.7002,68,"Hotel ""Drei Kreuz"""
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
125,jashpurnagar,IN,22.9000,84.1500,87,No hotel found
128,half moon bay,US,37.4636,-122.4286,86,Half Moon Bay Inn
146,puerto del rosario,ES,28.5004,-13.8627,73,Hotel Tamasite


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display
#could not make the map show
# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE